In [9]:
# universally modules
import re
import sys
import regex
import nltk.data
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append("../src")

# preprocessing and transformation modules
import fasttext
import Preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# model algorithm
from xgboost import XGBRegressor, XGBClassifier

# evaluation modules
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report

### Parameters

In [10]:
# to speed up the process choose a sample size to randomly draw a sample of the whole daataset
sample_size = 100_000

# remove all text that contain less than n chars
min_chars_per_text = 50

# which features will be used for the TF-IDF transformation
text_features = "text_preprocessed"

# set directory where data is stored

data_directory = "../../data/"

#### define the target variable and categorial variables used in later transformations ###
#### Case 1: gender 
#target_variable = "gender"
#categorial_variables =  ["topic", "sign"]

# Case 2: topic
#target_variable = "topic"
#categorial_variables =  ["gender", "sign"]

# Case 3: age
#target_variable = "age"
#categorial_variables =  ["topic", "gender", "sign"]

# Case 4: sign
#target_variable = "sign"
#categorial_variables =  ["gender", "topic"]

# Case 5: all
target_variable = "all"
categorial_variables =  ["topic", "gender", "sign"]
############################################################################################

# use only words that occur at least sqrt_3(X) times 
min_df_exponent = (1/3)

### Initialization 

In [11]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
df = pd.read_csv(data_directory + "blogtext.csv")

In [12]:
# draw random sample for faster processing:
df = df.sample(sample_size, random_state=42)
df

,id,gender,age,topic,sign,date,text
240436,1058543,female,24,indUnk,Sagittarius,"02,August,2004",1 Corinthians 5:11 'But now I have wri...
54139,3440336,female,23,Arts,Virgo,"08,July,2004",Having moved to Jersey City nearly a mo...
637911,3546243,female,24,indUnk,Taurus,"02,July,2004","urlLink Hey, Metro! Need To Save..."
61178,3367064,female,26,indUnk,Taurus,"29,July,2004",The most hilarious movie I have s...
314089,3590478,female,34,Technology,Capricorn,"20,April,2004",Short of being attached to the b...
...,...,...,...,...,...,...,...
25853,1939766,female,23,Arts,Aquarius,"25,January,2004",How did this happen? Yesterday I ...
34366,2781841,female,23,Student,Pisces,"07,May,2004","I've been thinking about language lately, w..."
177814,3567033,male,25,indUnk,Gemini,"01,July,2004",urlLink I was able to get up ...
606589,3353160,female,14,Arts,Leo,"07,June,2004","Sorry again about that, but i've be..."


### Filtering

In [13]:
# filter for a mininmal number of letters in a tweet:
df = df[df["text"].str.count(r"[a-zA-Z]") >= min_chars_per_text]
df = df.reset_index(drop=True)

### Feature Engineering

In [14]:
#def findDates(text):
#    try:
#        return len([date for date in\
#                    datefinder.find_dates(text)])
#    except:
#        return 0

In [15]:
def buildFeatures(text):
    text_split = text.split()
    len_text = len(text)
    sentence_split = tokenizer.tokenize(text)
    
    # find the number of urls in the text
    keywords = ["urlLink","http","www"]
    nb_urls = sum((any(keyword in pattern for keyword in keywords))\
               for pattern in text.split())
    # find the number of mails in the text
    nb_mails = len(re.findall(r"([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+|\bmail\b)"\
                      ,text))
    
    # find the number of dates in the text
#    nb_dates = findDates(text)
     
    # find characteristics about the usage of letters, numbers and symbols
    uppercase_ratio = len(re.findall(r'[A-Z]', text))/len_text
    lowercase_ratio = len(re.findall(r'[a-z]', text))/len_text
    number_ratio = len(re.findall(r'[0-9]', text))/len_text
    symbol_ratio = len(re.findall(r'[$-/:-?{-~!"^_`\[\]]', text))/len_text

    # find characteristics about the letters per word
    sentence_len_word = [len(word) for word in text_split]
    avg_letters_per_word = np.mean([len(word) for word in text_split])
    var_letters_per_word = np.var([len(word) for word in text_split])
    unique_words_ratio = len(set(text_split))/len(text_split)

    # find characteristics about the letters per sentence
    sentence_len_list = [len(sentence) for sentence in sentence_split]
    avg_letters_per_sentence = np.mean(sentence_len_list)
    var_letters_per_sentence = np.var(sentence_len_list)
    
    # find characteristics about the words per sentence
    words_per_sentence_len_list = [len(sentence.split()) for sentence in sentence_split]
    avg_words_per_sentence = np.mean(words_per_sentence_len_list)
    var_words_per_sentence = np.var(words_per_sentence_len_list)
    
    # find the trumps
    uppercase_per_sentence_ratio = [len(re.findall(r'[A-Z]', sentence))/len(sentence)\
                                    for sentence in sentence_split]
    max_sentence_uppercase_ratio = max(uppercase_per_sentence_ratio)
    max_sentence_uppercase_len = len(sentence_split[uppercase_per_sentence_ratio.index(max_sentence_uppercase_ratio)])
    
    return len_text, nb_urls, nb_mails,\
           uppercase_ratio, lowercase_ratio, number_ratio, symbol_ratio,\
           avg_letters_per_word, var_letters_per_word, unique_words_ratio,\
           avg_letters_per_sentence, var_letters_per_sentence,\
           avg_words_per_sentence, var_words_per_sentence,\
           max_sentence_uppercase_ratio, max_sentence_uppercase_len        
           

In [16]:
# append the data
features = [buildFeatures(text) for text  in tqdm(df["text"])]

# append the data
columns = ["Text length", "Number URLs", "Number mails",\
          "Uppercase ratio", "Lowercase ratio", "Number ratio", "Symbol ratio",\
          "Average letters per word", "Variance of letters per word", "Unique words ratio",\
          "Average letters per sentence", "Variance of letters per sentence",\
          "Average words per sentence", "Variance of words per sentence",\
          "Maximal uppercase ratio per sentence", "Length of the maximal uppercase ratio sentence"]

# merge the features with the original dataset
df_preprocessed = df.merge(pd.DataFrame(features, columns=columns), left_index=True, right_index=True)

100%|██████████| 90439/90439 [01:42<00:00, 885.30it/s] 


### Text Preprocessing

In [17]:
# use the preprocessing  module
preprocessor = Preprocessing.Preprocessing()
df_preprocessed["text_preprocessed"] = preprocessor.ProcessMany(df_preprocessed["text"])

# predict the main language
model = fasttext.load_model('../src/data/lid.176.ftz')
df_preprocessed["main_language"] = [model.predict(text)[0][0].split("__")[-1] for text in tqdm(df_preprocessed["text_preprocessed"])]

100%|██████████| 90439/90439 [00:14<00:00, 6430.74it/s]


In [18]:
def remove_non_ascii_words(text):
    return "".join([f"{word} " for word in text.split() if len(re.findall(r'[^\x00-\x7f]', word)) == 0])


In [19]:
# drop unnecassary features
df_filtered = df_preprocessed[(df_preprocessed["main_language"] == "en")]\
                .drop(["id","text","date","main_language"], axis= 1)

# remove all words with non-ascii chars
df_filtered["text_preprocessed"] = df_filtered["text_preprocessed"].apply(remove_non_ascii_words)

In [20]:
# Save the preprocessed data before removing the "original" taxt, to interpret the clusters later
df_filtered.to_csv(data_directory + f'bolgtext_filtered.csv', index = False)
df_filtered

,gender,age,topic,sign,Text length,Number URLs,Number mails,Uppercase ratio,Lowercase ratio,Number ratio,...,Average letters per word,Variance of letters per word,Unique words ratio,Average letters per sentence,Variance of letters per sentence,Average words per sentence,Variance of words per sentence,Maximal uppercase ratio per sentence,Length of the maximal uppercase ratio sentence,text_preprocessed
0,female,24,indUnk,Sagittarius,826,0,0,0.024213,0.713075,0.004843,...,4.082803,5.642825,0.687898,66.166667,3770.972222,13.083333,128.409722,0.166667,6,1 corinthians 5:11 ' but -pron- write company ...
1,female,23,Arts,Virgo,2666,0,0,0.028132,0.746437,0.002626,...,4.378049,5.804234,0.623984,105.120000,3962.745600,19.680000,124.777600,0.133333,15,move jersey city a month ago -pron- idea -pron...
2,female,24,indUnk,Taurus,269,1,0,0.040892,0.665428,0.000000,...,4.545455,6.247934,0.909091,50.600000,1954.640000,8.800000,84.960000,0.210526,19,hey metro save money a hike heel fare hike yea...
3,female,26,indUnk,Taurus,1756,0,0,0.015945,0.763667,0.000000,...,4.580645,6.153174,0.632258,78.227273,1352.721074,14.090909,45.900826,0.055556,18,hilarious movie -pron- year a comedy movie abs...
4,female,34,Technology,Capricorn,9046,0,0,0.037917,0.742096,0.003648,...,4.719287,6.343224,0.550605,112.936709,4624.844095,19.898734,144.420125,0.333333,60,short attach bumper a pickup truck a cock ring...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90434,female,23,Arts,Aquarius,609,0,0,0.032841,0.715928,0.000000,...,4.157895,5.483841,0.666667,59.100000,1282.090000,11.400000,43.240000,0.105263,19,happen yesterday -pron- mall store -pron- visi...
90435,female,23,Student,Pisces,4006,5,0,0.020969,0.742636,0.009735,...,4.484722,8.055322,0.469444,98.200000,5066.410000,18.000000,182.650000,0.125000,8,-pron- ' ve think language good -pron- ' m maj...
90436,male,25,indUnk,Gemini,361,2,0,0.022161,0.709141,0.000000,...,4.138462,5.534675,0.723077,116.666667,402.888889,21.666667,29.555556,0.040000,125,-pron- fish poudre river a saturday morning -p...
90437,female,14,Arts,Leo,392,0,0,0.020408,0.701531,0.000000,...,4.140845,4.261853,0.845070,74.200000,1987.760000,14.200000,51.760000,0.062500,48,i ' ve shut room past day live water completel...


### Transformation

In [21]:
class StackedTransformation:
    
    def  __init__(self, X=None, y=None, numerical_transformer=None, text_transformer=None, text_features=None):
        self.X = X
        self.y = y
        self.numerical_transformer = numerical_transformer
        self.text_transformer = text_transformer
        self.text_features = text_features
        
    def data_split(self, X, y, test_size= 0.2):
        self.X_train, self.X_test, self.y_train, self.y_test = X, X, y, y
        return self.X_train, self.X_test, self.y_train, self.y_test
        
    def build_transformer(self, X=None, y=None,\
                          numerical_transformer=None, text_transformer=None, text_features=None):
        # check if variables are present
        if X == None:
            X = self.X
        if y == None:
            y = self.y
        if numerical_transformer == None:
            numerical_transformer = self.numerical_transformer
        if text_transformer == None:
            text_transformer = self.text_transformer
        if text_features == None:
            text_features = self.text_features

        # split the data
        
        self.data_split(X, y)
        
        # Create datasets for each classifier
        self.X_train_text = self.X_train[text_features]
        self.X_test_text =  self.X_test[text_features]
        
        self.X_train_numerical = self.X_train.drop(text_features, axis=1)
        self.X_test_numerical = self.X_test.drop(text_features, axis=1)

        # create transformers 
        self.numerical_transformer = numerical_transformer
        self.text_transformer = text_transformer
        
        #create transformed training batches
        self.X_train_numerical_transformed = self.numerical_transformer.fit_transform(self.X_train_numerical)
        self.X_test_numerical_transformed = self.numerical_transformer.transform(self.X_test_numerical)

        self.X_train_text_transformed = self.text_transformer.fit_transform(self.X_train_text)
        self.X_test_text_transformed = self.text_transformer.transform(self.X_test_text)

    def transform_one(self, text):
        entry = pd.DataFrame(columns=text.keys())
        entry = entry.append(text, ignore_index=True)
        
        # create datasets for each transformer
        entry_text = entry[text_features]
        entry_numerical = entry.drop(text_features, axis=1)
        
        entry_numerical_transformed = self.numerical_transformer.transform(entry_numerical)
        entry_text_transformed = self.text_transformer.transform(entry_text)
        
        return {"transformed_text": entry_text_transformed,\
               "transformed_numerical": entry_numerical_transformed}

### Data Split

In [22]:
#X,y = df_filtered.drop(target_variable, axis=1),df_filtered[target_variable]
X,y = df_filtered,df_filtered["gender"]

In [23]:
# use the  text transformer class to create two transformers for the textual and the numerical model
text_transformer = TfidfVectorizer(ngram_range=(1,1), min_df=int(len(X)**(min_df_exponent)))
print('text transformer finished')

numerical_transformer = make_column_transformer((OneHotEncoder(handle_unknown="ignore"), categorial_variables)\
                                                       , remainder=StandardScaler())
print('numerical transformer finished')

stacking = StackedTransformation(X, y, numerical_transformer, text_transformer, text_features)
print('stacking created')

stacking.build_transformer()
print('stacking finished')

text transformer finished
numerical transformer finished
stacking created
stacking finished


### Clustering

In [24]:
text_features = stacking.text_transformer.get_feature_names()
text_data = stacking.X_train_text_transformed.toarray()

df_text_cluster = pd.DataFrame(text_data, columns=text_features)
df_text_cluster

# Die Features beschreiben die Worte im Text
# Die Werte sind die TF*IDF-transformierten Textdaten

,00,000,00am,00pm,01,02,03,04,05,06,...,zen,zeppelin,zero,zip,zodiac,zoe,zombie,zone,zoo,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033552,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
88386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
88387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
88388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [25]:
try:
    numerical_data = stacking.X_train_numerical_transformed.toarray()
except:
    numerical_data = stacking.X_train_numerical_transformed


numerical_features = np.append(stacking.numerical_transformer.transformers_[0][1].get_feature_names(),\
                     stacking.X_train_numerical.columns.drop(categorial_variables))

df_numerical_cluster = pd.DataFrame(numerical_data, columns=numerical_features)
df_numerical_cluster


# Die Features mit den x0 - xi Werten beschreiben die Ausprägungen die kategorialen Variablen
# das jeweilige i beschreibt das i-te Element der im Punkt "Target Variable" definierten liste categorial_variables
# Die verbleibenden Features (ohne xi) sind Standardskaliert, (x - \mu)/\sigma

,x0_Accounting,x0_Advertising,x0_Agriculture,x0_Architecture,x0_Arts,x0_Automotive,x0_Banking,x0_Biotech,x0_BusinessServices,x0_Chemicals,...,Symbol ratio,Average letters per word,Variance of letters per word,Unique words ratio,Average letters per sentence,Variance of letters per sentence,Average words per sentence,Variance of words per sentence,Maximal uppercase ratio per sentence,Length of the maximal uppercase ratio sentence
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.406886,-0.185619,-0.024471,-0.381103,-0.291581,-0.046192,-0.244946,-0.048985,-0.002707,-0.432317
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.528082,-0.066584,-0.023874,-0.837267,-0.045730,-0.044673,-0.011748,-0.050032,-0.167852,-0.383229
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.280450,0.000909,-0.022233,1.197579,-0.389829,-0.060585,-0.396366,-0.061504,0.214590,-0.361412
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.594240,0.015097,-0.022583,-0.778212,-0.215462,-0.065355,-0.209328,-0.072757,-0.553193,-0.366866
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.384038,0.070993,-0.021881,-1.360983,0.003604,-0.039426,-0.004016,-0.044372,0.823022,-0.137788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88385,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.574430,-0.155344,-0.025058,-0.532634,-0.336182,-0.065915,-0.304453,-0.073524,-0.306923,-0.361412
88386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.453092,-0.023577,-0.015550,-1.940234,-0.089405,-0.035927,-0.071138,-0.033358,-0.209139,-0.421408
88387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.180529,-0.163179,-0.024871,-0.130027,0.027146,-0.072882,0.058482,-0.077467,-0.630261,0.216737
88388,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.632138,-0.162218,-0.029577,0.740656,-0.240880,-0.060323,-0.205471,-0.071069,-0.518787,-0.203239


In [26]:
df_text_cluster.to_csv(data_directory + f'Data_Text_Clustering.csv', index = False)
df_numerical_cluster.to_csv(data_directory + f'Data_Numerical_Clustering.csv', index = False)